In [ ]:
import glob
import math
import uproot
import numpy as np
import pandas as pd
import pkgutil
import uproot_methods

import functions_giammi as fugi
from importlib import reload

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
%matplotlib inline

import re

import mplhep as hep
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import itertools

from __future__ import division

plt.style.use(hep.style.ROOT) # For now ROOT defaults to CMS

#loads the temperaure cmap like Philipp for both matpplotlib and plotly
cmap_temp, cmap_temp_go, Magma_r, Seismic_r = fugi.customcmaps()

#to see which classes have been defined so far
[modname for importer, modname, ispkg in pkgutil.walk_packages(uproot_methods.classes.__path__)]

fileR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R_enant_scan25step.root")
fileS = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S_enant_scan25step.root")

PATH_CH = ["angular_distr_el/CH11-2-3_m2x-m29/","angular_distr_el/CH9-10_m1x-m42/"]
DIR = ["map_AVG_ph_En/","map_ScanEn_SHORT/","map_ScanEn_LONG/"]
EN = ["NO_gate/","700/","350/","20/"]
EN_opt = 1
# the next layer is L or R, selected later
ID_AVG=["PECD_el(theta)","PECD_el(theta)_redPHI"]
ID_SHORT=["cos(theta)_vs_pEn","cos(theta)_vs_pEn_pol","cos(theta)_vs_pEn_redPHI","cos(theta)_vs_pEn_pol_redPHI"]
ID_PECD=["PECD_el(theta)_@534.5eV_MINUS","PECD_el(theta)_@535.5eV_PLUS","PECD_el(theta)_@534.5eV_MINUS_redPHI","PECD_el(theta)_@535.5eV_PLUS_redPHI"]

loc_avg=PATH_CH[0]+DIR[0]+EN[EN_opt]
loc_short=PATH_CH[0]+DIR[1]+EN[EN_opt]
loc_long=PATH_CH[0]+DIR[2]+EN[EN_opt]

In [ ]:
#commands for uproot single graphs loading
fileR[loc_short+"R/"+ID_SHORT[0]].all_members
# fileR[loc_short+"R/"+ID_SHORT[0]].member("fXaxis") #just the object
fileR[loc_short+"R/"+ID_SHORT[0]].axis("x").centers() #N
# fileR[loc_short+"R/"+ID_SHORT[0]].axis("x").edges() #N+1
# fileR[loc_short+"R/"+ID_SHORT[0]].to_numpy()

In [ ]:
#cos(theta) vs energy
ID_sel=1 #0 normal, 1 pol plane, 2 redpphi, 3 redphipolplane
en=fileR[loc_short+"R/"+ID_SHORT[ID_sel]].axis("x").centers() #N
ctheta=fileR[loc_short+"R/"+ID_SHORT[ID_sel]].axis("y").centers() #N

cthetaEN_R=fileR[loc_short+"R/"+ID_SHORT[ID_sel]].to_numpy()[0]
cthetaEN_R_err=fileR[loc_short+"R/"+ID_SHORT[ID_sel]].errors() #to be normalised too
cthetaEN_R_norm,cthetaEN_R_err_norm=fugi.normalise_with_err(cthetaEN_R,cthetaEN_R_err,normtype=0)

cthetaEN_L=fileR[loc_short+"L/"+ID_SHORT[ID_sel]].to_numpy()[0]
cthetaEN_L_err=fileR[loc_short+"L/"+ID_SHORT[ID_sel]].errors() #to be normalised too
cthetaEN_L_norm,cthetaEN_L_err_norm=fugi.normalise_with_err(cthetaEN_L,cthetaEN_L_err,normtype=0)

print(cthetaEN_R.shape)
print(ctheta.shape)
print(en.shape)

In [ ]:
#PECD between two diffrent helicities
cPECD_en =  np.divide(np.subtract(cthetaEN_R_norm[3:],cthetaEN_L_norm[3:]),np.add(cthetaEN_R_norm[3:],cthetaEN_L_norm[3:]))
errPECD_en = fugi.error_calc(cthetaEN_R_norm[3:],cthetaEN_L_norm[3:],cthetaEN_R_err_norm[3:],cthetaEN_L_err_norm[3:])

In [ ]:
#534-535eV classic PECD diatomic
# standard 0-1
# redPHI 2-3
cbeta=fileR[loc_long+"R/"+ID_PECD[2]].axis("x").centers() #N
cthetaM=fileR[loc_long+"R/"+ID_PECD[2]].axis("y").centers() #N

MFPAD_534_R=fileR[loc_long+"R/"+ID_PECD[2]].to_numpy()[0]
MFPAD_534_R_err=fileR[loc_long+"R/"+ID_PECD[2]].errors() #to be normalised too
MFPAD_534_R_norm,MFPAD_534_R_err_norm=fugi.normalise_with_err(MFPAD_534_R,MFPAD_534_R_err,normtype=0)
ctheta_534_R=np.sum(MFPAD_534_R, axis=1)
ctheta_534_R_err=np.sum(MFPAD_534_R_err, axis=1)
ctheta_534_R_norm, ctheta_534_R_norm_err = fugi.normalise_with_err(ctheta_534_R,ctheta_534_R_err)


MFPAD_535_R=fileR[loc_long+"R/"+ID_PECD[3]].to_numpy()[0]
MFPAD_535_R_err=fileR[loc_long+"R/"+ID_PECD[3]].errors() #to be normalised too
MFPAD_535_R_norm,MFPAD_535_R_err_norm=fugi.normalise_with_err(MFPAD_535_R,MFPAD_535_R_err,normtype=0)
ctheta_535_R=np.sum(MFPAD_535_R, axis=1)
ctheta_535_R_err=np.sum(MFPAD_535_R_err, axis=1)
ctheta_535_R_norm, ctheta_535_R_norm_err = fugi.normalise_with_err(ctheta_535_R,ctheta_535_R_err)

MFPAD_534_L=fileR[loc_long+"L/"+ID_PECD[2]].to_numpy()[0]
MFPAD_534_L_err=fileR[loc_long+"L/"+ID_PECD[2]].errors() #to be normalised too
MFPAD_534_L_norm,MFPAD_534_L_err_norm=fugi.normalise_with_err(MFPAD_534_L,MFPAD_534_L_err,normtype=0)
ctheta_534_L=np.sum(MFPAD_534_L, axis=1)
ctheta_534_L_err=np.sum(MFPAD_534_L_err, axis=1)
ctheta_534_L_norm, ctheta_534_L_norm_err = fugi.normalise_with_err(ctheta_534_L,ctheta_534_L_err)

MFPAD_535_L=fileR[loc_long+"L/"+ID_PECD[3]].to_numpy()[0]
MFPAD_535_L_err=fileR[loc_long+"L/"+ID_PECD[3]].errors() #to be normalised too
MFPAD_535_L_norm,MFPAD_535_L_err_norm=fugi.normalise_with_err(MFPAD_535_L,MFPAD_535_L_err,normtype=0)
ctheta_535_L=np.sum(MFPAD_535_L, axis=1)
ctheta_535_L_err=np.sum(MFPAD_535_L_err, axis=1)
ctheta_535_L_norm, ctheta_535_L_norm_err = fugi.normalise_with_err(ctheta_535_L,ctheta_535_L_err)

print(MFPAD_534_R.shape)
print(ctheta_534_R_norm.shape)

In [ ]:
#PECD between two diffrent helicities
PECD_Auger_534 =  np.divide(np.subtract(MFPAD_534_R_norm,MFPAD_534_L_norm),np.add(MFPAD_534_R_norm,MFPAD_534_L_norm))
errPECD_Auger_534 = fugi.error_calc(MFPAD_534_R_norm,MFPAD_534_L_norm,MFPAD_534_R_err_norm,MFPAD_534_L_err_norm)

PECD_Auger_535 =  np.divide(np.subtract(MFPAD_535_R_norm,MFPAD_535_L_norm),np.add(MFPAD_535_R_norm,MFPAD_535_L_norm))
errPECD_Auger_535 = fugi.error_calc(MFPAD_535_R_norm,MFPAD_535_L_norm,MFPAD_535_R_err_norm,MFPAD_535_L_err_norm)

#PECD cos projection
cPECD_534 =  np.divide(np.subtract(ctheta_534_R_norm,ctheta_534_L_norm),np.add(ctheta_534_R_norm,ctheta_534_L_norm))
errcPECD_534 = fugi.error_calc(ctheta_534_R_norm,ctheta_534_L_norm,ctheta_534_R_norm_err,ctheta_534_L_norm_err)*0.1

cPECD_535 =  np.divide(np.subtract(ctheta_535_R_norm,ctheta_535_L_norm),np.add(ctheta_535_R_norm,ctheta_535_L_norm))
errcPECD_535 = fugi.error_calc(ctheta_535_R_norm,ctheta_535_L_norm,ctheta_535_R_norm_err,ctheta_535_L_norm_err)*0.1
#NOTE: the adjustmnet is empirical

In [ ]:
fig,(ax1,ax2)= plt.subplots(1,2, figsize=(25, 10), sharey=True)
plt.suptitle("ctehta vs EN "+EN[EN_opt])
cs1=ax1.contourf(en[3:],ctheta,cthetaEN_R_norm[3:].T, cmap=cmap_temp, levels=15)
ax1.set_title("R enant")
ax1.set_xlabel('photon en [eV]')
ax1.set_ylabel('cos(theta) [adm]')
# ax1.set_aspect(aspect)
cbar1=fig.colorbar(cs1,extend='both', shrink=0.9, ax=ax1)
cbar1.set_ticklabels(["min","max"])
cbar1.set_ticks([cthetaEN_R_norm[3:].min(),cthetaEN_R_norm[3:].max()])


cs2=ax2.contourf(en[3:],ctheta,cthetaEN_L_norm[3:].T, cmap=cmap_temp, levels=15)
ax2.set_title("L enant")
ax2.set_xlabel('photon en [eV]')
ax2.set_ylabel('cos(theta) [adm]')
# ax2.set_aspect(aspect)
cbar2=fig.colorbar(cs2,extend='both', shrink=0.9, ax=ax2)
cbar2.set_ticklabels(["min","max"])
cbar2.set_ticks([cthetaEN_L_norm[3:].min(),cthetaEN_L_norm[3:].max()])

# fig.savefig("../PYTHON_graphs/OUTPUTS/RCX_MFPAD_norm0_comparison_"+CH+"_"+en+"_exp.png", figsize=(50, 24), dpi=144, transparent=False)
# plt.show()

In [ ]:
fig,ax= plt.subplots(1, figsize=(12, 10))
cs=ax.contourf(en[3:],ctheta,cPECD_en.T, cmap=cmap_temp, levels=20)
ax.set_title("PECD vs energy "+EN[EN_opt])
# aspect=(phiM.max()/cosM.max())
ax.set_xlabel('photon en [eV]')
ax.set_ylabel('cos(theta) [adm]')
cbar = fig.colorbar(cs,extend='both',shrink=0.9, ax=ax)
# cbar.set_ticklabels(["min","max"])
# cbar.set_ticks([cPECD_en.min(),cPECD_en.max()])
cbar.ax.set_ylabel('% of asymmetry')
# ax.set_aspect(aspect)
# fig.savefig("../PYTHON_graphs/OUTPUTS/SCR_MFPAD_singleNONORM_"+CH+"_"+en+"_exp.png", figsize=(50, 24), dpi=144, transparent=False)

In [ ]:
import functions_minuit as fumi
# param_matrix=np.zeros((6,72)) # alternative nparray initialization
param_matrix_cos = [[0 for i in range(6)] for j in range(21)] #i col, j row

# fig, axes = plt.subplots(3,7, figsize=(50, 24), constrained_layout=True, sharex=True , sharey=True)
fig, axes = plt.subplots(3,7, figsize=(25, 12), constrained_layout=True, sharex=True , sharey=True)
custom_xlim = (-1, 1)
custom_ylim = (-0.1,0.1) #custom_ylim = (-1,1)

# limits=[(-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1)]
limits=[(-1, 1), (-1, 2), (None, None), (None, None), (None, None), (None, None)]

for i,el,el_err,ax in zip(range(21),cPECD_en,errPECD_en,axes.flatten()): #fixe because of the photon directions, equal to range(len(cosphi))
    
    x_data = ctheta;y_data = el;y_err = el_err
    #non sorted as test
    # x_data = ctheta_axis_red;y_data = cPECD_std[i];y_err = errPECD_std[i]
    
    m=fumi.iminuit_err(x_data,y_data,y_err,limits)
    fumi.plotminuit(x_data,y_data,y_err,m,ax,custom_xlim,custom_ylim)

    for counter,el in enumerate(m.parameters): # len = 6
        param_matrix_cos[i][counter]=((m.values[el], m.errors[el]))

param_matrix_cos=np.array(param_matrix_cos) #numpy tranformation!!!

# fugi.overlaygraph(fig,wspace=0.08, hspace=0.08)
plt.show() 

In [ ]:
fig,(ax1,ax2)= plt.subplots(1,2, figsize=(25, 10))
plt.suptitle("b1 and b2 vs photon energy "+EN[EN_opt])
ax1.scatter(en[3:],param_matrix_cos[:,0,0])
ax1.errorbar(en[3:],param_matrix_cos[:,0,0], param_matrix_cos[:,0,1], fmt="o")
ax1.axhline(y=0, color='k', linestyle='--', alpha=0.7)
ax1.set_xlabel('en [eV]')
ax1.set_ylabel('b1 [adm]')
# ax1.set_aspect(aspect)

ax2.scatter(en[3:],param_matrix_cos[:,1,0])
ax2.errorbar(en[3:],param_matrix_cos[:,1,0], param_matrix_cos[:,1,1], fmt="o")
ax2.axhline(y=-1, color='r', linestyle='--', alpha=0.7)
ax2.axhline(y=2, color='r', linestyle='--', alpha=0.7)
ax2.set_xlabel('en [eV]')
ax2.set_ylabel('b2 [adm]')
# ax2.set_aspect(aspect)

# fig.savefig("../PYTHON_graphs/OUTPUTS/RCX_MFPAD_norm0_comparison_"+CH+"_"+en+"_exp.png", figsize=(50, 24), dpi=144, transparent=False)
# plt.show()

In [ ]:
fig,ax= plt.subplots(1, figsize=(12, 10))
cs=ax.contourf(cbeta,ctheta,PECD_Auger_534.T, cmap=cmap_temp, levels=20)
ax.set_title("Contrast map 534 eV "+EN[EN_opt])
# aspect=(phiM.max()/cosM.max())
ax.set_xlabel('cos(beta) [adm]')
ax.set_ylabel('cos(theta) [adm]')
cbar = fig.colorbar(cs,extend='both',shrink=0.9, ax=ax)
cbar.ax.set_ylabel('% of asymmetry')
# ax.set_aspect(aspect)
# fig.savefig("../PYTHON_graphs/OUTPUTS/SCR_MFPAD_singleNONORM_"+CH+"_"+en+"_exp.png", figsize=(50, 24), dpi=144, transparent=False)

In [ ]:
#INTESNITY OF THE COS(THETA)
import functions_minuit as fumi

fig, ax = plt.subplots()
custom_xlim = (-1, 1)
    
limits=[(None, None), (-1, 1), (-1, 2)]

x_data = cthetaM;y_data = ctheta_534_R;y_err = ctheta_534_R_err

m=fumi.iminuit_err_cos(x_data,y_data,y_err,limits)
fumi.plotminuit_cos(x_data,y_data,y_err,m,ax,custom_xlim)
ax.set_title("Intensity c(theta) R "+EN[EN_opt])
ax.set_xlabel('cos(theta) [adm]')
ax.set_ylabel('counts')

fit_info = []
for p, v, e in zip(m.parameters, m.values, m.errors):
    fit_info.append(f"{p} = ${v:.3f} \\pm {e:.3f}$")

ax.legend(title="\n".join(fit_info), loc="best")

# aspect=(custom_xlim[1]/custom_ylim[1])
# ax.set_aspect(aspect)

m
# fig.savefig("../PYTHON_graphs/OUTPUTS/Diatomic/cos(th)x_combi_"+CH+"_"+en+"_exp.png", figsize=(12, 10), dpi=144, transparent=False)

In [ ]:
#PECD INTERPOLATION RANDOM
fig, ax = plt.subplots()
custom_xlim = (-1, 1)
custom_ylim = (-0.05, 0.05)
    
# limits=[(-1, 1), (-1, 1),(-1, 1), (-1, 1),(-1, 1), (-1, 1)] #PECD6
limits=[(-1, 1), (-1, 2)] #PECD2

x_data = cthetaM;y_data = cPECD_534;y_err = errcPECD_534

m=fumi.iminuit_err2(x_data,y_data,y_err,limits)
fumi.plotminuit2(x_data,y_data,y_err,m,ax,custom_xlim, ylim=custom_ylim)

ax.set_title("PECD random ")
ax.set_xlabel('cos(theta) [adm]')
ax.set_ylabel('PECD %')

fit_info = []
for p, v, e in zip(m.parameters, m.values, m.errors):
    fit_info.append(f"{p} = ${v:.3f} \\pm {e:.3f}$")

ax.legend(title="\n".join(fit_info), loc="best")

aspect=(custom_xlim[1]/custom_ylim[1])
ax.set_aspect(aspect)

m
# fig.savefig("../PYTHON_graphs/OUTPUTS/Diatomic/PECD_random_"+CH+"_"+en+"_exp.png", figsize=(12, 10), dpi=144, transparent=False)

In [ ]:
fig,ax= plt.subplots(1, figsize=(12, 10))
cs=ax.contourf(cbeta,cthetaM,PECD_Auger_535.T, cmap=cmap_temp, levels=20)
ax.set_title("Contrast map 535 eV "+EN[EN_opt])
# aspect=(phiM.max()/cosM.max())
ax.set_xlabel('cos(beta) [adm]')
ax.set_ylabel('cos(theta) [adm]')
cbar = fig.colorbar(cs,extend='both',shrink=0.9, ax=ax)
cbar.ax.set_ylabel('% of asymmetry')
# ax.set_aspect(aspect)
# fig.savefig("../PYTHON_graphs/OUTPUTS/SCR_MFPAD_singleNONORM_"+CH+"_"+en+"_exp.png", figsize=(50, 24), dpi=144, transparent=False)

In [ ]:
#INTESNITY OF THE COS(THETA)
#COMBINED ELLICITIES
import functions_minuit as fumi

fig, ax = plt.subplots()
custom_xlim = (-1, 1)
    
limits=[(None, None), (-1, 1), (-1, 2)]

x_data = cthetaM;y_data = ctheta_535_R;y_err = ctheta_535_R_err

m=fumi.iminuit_err_cos(x_data,y_data,y_err,limits)
fumi.plotminuit_cos(x_data,y_data,y_err,m,ax,custom_xlim)
# ax.set_title("Intensity c(theta) combi "+CH+" "+en)
ax.set_xlabel('cos(theta) [adm]')
ax.set_ylabel('normalised counts')

fit_info = []
for p, v, e in zip(m.parameters, m.values, m.errors):
    fit_info.append(f"{p} = ${v:.3f} \\pm {e:.3f}$")

ax.legend(title="\n".join(fit_info), loc="best")

# aspect=(custom_xlim[1]/custom_ylim[1])
# ax.set_aspect(aspect)

m
# fig.savefig("../PYTHON_graphs/OUTPUTS/Diatomic/cos(th)x_combi_"+CH+"_"+en+"_exp.png", figsize=(12, 10), dpi=144, transparent=False)

In [ ]:
#PECD INTERPOLATION RANDOM
fig, ax = plt.subplots()
custom_xlim = (-1, 1)
custom_ylim = (-0.05, 0.05)
    
# limits=[(-1, 1), (-1, 1),(-1, 1), (-1, 1),(-1, 1), (-1, 1)] #PECD6
limits=[(-1, 1), (-1, 2)] #PECD2

x_data = cthetaM;y_data = cPECD_535;y_err = errcPECD_535

m=fumi.iminuit_err2(x_data,y_data,y_err,limits)
fumi.plotminuit2(x_data,y_data,y_err,m,ax,custom_xlim, ylim=custom_ylim)

ax.set_title("PECD random ")
ax.set_xlabel('cos(theta) [adm]')
ax.set_ylabel('PECD %')

fit_info = []
for p, v, e in zip(m.parameters, m.values, m.errors):
    fit_info.append(f"{p} = ${v:.3f} \\pm {e:.3f}$")

ax.legend(title="\n".join(fit_info), loc="best")

aspect=(custom_xlim[1]/custom_ylim[1])
ax.set_aspect(aspect)

m
# fig.savefig("../PYTHON_graphs/OUTPUTS/Diatomic/PECD_random_"+CH+"_"+en+"_exp.png", figsize=(12, 10), dpi=144, transparent=False)